# Day 11: Reactor

## Part 1

The problem asks us to find the number of distinct paths from a starting node `you` to an ending node `out` in a directed graph. The graph is defined by a list of devices and their outputs.

### Approach

1.  **Parse the Input**: Read the input file and build an adjacency list representing the graph.
2.  **Count Paths**: Since we need to count the number of paths, and the graph is likely a Directed Acyclic Graph (DAG) (implied by "data... can't flow backwards"), we can use a recursive Depth First Search (DFS) with memoization.
    *   `count_paths(current_node, target_node)`:
        *   If `current_node == target_node`, return 1.
        *   If `current_node` is already in our memoization table, return the stored value.
        *   Otherwise, sum `count_paths(neighbor, target_node)` for all neighbors of `current_node`.
        *   Store and return the sum.


In [ ]:
import collections

def parse_input(filename):
    graph = collections.defaultdict(list)
    with open(filename, 'r') as f:
        for line in f:
            parts = line.strip().split(': ')
            node = parts[0]
            if len(parts) > 1:
                neighbors = parts[1].split(' ')
                graph[node] = neighbors
            else:
                graph[node] = []
    return graph

def count_paths(graph, start, end, memo=None):
    if memo is None:
        memo = {}
    
    if start == end:
        return 1
    
    if start in memo:
        return memo[start]
    
    total_paths = 0
    if start in graph:
        for neighbor in graph[start]:
            total_paths += count_paths(graph, neighbor, end, memo)
    
    memo[start] = total_paths
    return total_paths


In [ ]:
test_graph = parse_input('test.txt')
test_paths = count_paths(test_graph, 'you', 'out')
print(f"Test paths: {test_paths}")
assert test_paths == 5


In [ ]:
graph = parse_input('input.txt')
part1_paths = count_paths(graph, 'you', 'out')
print(f"Part 1 Answer: {part1_paths}")


## Part 2

We need to find the number of paths from `svr` to `out` that visit both `dac` and `fft`.

Since the graph is a DAG, for a single path to visit both nodes, one must appear before the other in the path.
There are two cases:
1.  `svr` -> ... -> `dac` -> ... -> `fft` -> ... -> `out`
2.  `svr` -> ... -> `fft` -> ... -> `dac` -> ... -> `out`

We can calculate the number of paths for each segment and multiply them:
*   Case 1: `count_paths('svr', 'dac') * count_paths('dac', 'fft') * count_paths('fft', 'out')`
*   Case 2: `count_paths('svr', 'fft') * count_paths('fft', 'dac') * count_paths('dac', 'out')`

The total is the sum of these two cases. Note that if one node is not reachable from the other, the count for that segment will be 0, correctly handling the impossibility.


In [ ]:
# Case 1: svr -> dac -> fft -> out
case1 = count_paths(graph, 'svr', 'dac') * count_paths(graph, 'dac', 'fft') * count_paths(graph, 'fft', 'out')

# Case 2: svr -> fft -> dac -> out
case2 = count_paths(graph, 'svr', 'fft') * count_paths(graph, 'fft', 'dac') * count_paths(graph, 'dac', 'out')

part2_paths = case1 + case2
print(f"Part 2 Answer: {part2_paths}")
